#### **Setup Python**

In [41]:
# Importação - Bibliotecas Python
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import warnings
import unicodedata

# Remoção de avisos do Python
warnings.filterwarnings("ignore")

#### **Variáveis - Data Preprocessing TOTVS**

In [3]:
# Variável que armazena o caminho (path) das bases de dados presentes na camada raw
path_raw = "../data/raw"

# Variável que armazena o caminho (path) da camada "processed" onde as bases de dados serão armazenadas
path_processed = "../data/processed"

#### **Funções - Data Preprocessing TOTVS**

In [ ]:
# Função para padronizar strings: maiúsculas, remover acentos e espaços extras
def padronizar_texto(coluna):
    return (
        coluna.astype(str)
              .str.strip()
              .str.upper()
              .apply(lambda x: unicodedata.normalize("NFKD", x)
                      .encode("ascii", errors="ignore").decode("utf-8"))
    )

# Função para calcular a idade em meses
def idade_contrato_meses(data_assinatura):
    if pd.isnull(data_assinatura):
        return np.nan
    delta = relativedelta(pd.Timestamp.today(), data_assinatura)
    return delta.years * 12 + delta.months

#### **Processamento dos Dados - TOTVS**

##### **Pré-processamento: dados_clientes.csv**

In [ ]:
# Leitura da base de dados "dados_clientes.csv"
clientes = pd.read_csv(f"{path_raw}/dados_clientes.csv", sep=";")

# Cópia da base mãe
df_clientes = clientes.copy(deep=True)

# Aplicar nas colunas categóricas principais
colunas_categoricas = [
    "DS_PROD", "DS_LIN_REC", "CIDADE", "DS_CNAE", "DS_SEGMENTO",
    "DS_SUBSEGMENTO", "FAT_FAIXA", "MARCA_TOTVS", "MODAL_COMERC",
    "PERIODICIDADE", "SITUACAO_CONTRATO", "UF"
]

for col in colunas_categoricas:
    df_clientes[col] = padronizar_texto(df_clientes[col])

# Preenchimento de colunas com poucos nulos com 'NAO INFORMADO'
colunas_para_preencher = {
    "DS_SUBSEGMENTO": "NAO INFORMADO",
    "MARCA_TOTVS": "NAO INFORMADO",
    "MODAL_COMERC": "NAO INFORMADO",
    "PERIODICIDADE": "NAO INFORMADO",
    "SITUACAO_CONTRATO": "NAO INFORMADO"
}

for col, valor in colunas_para_preencher.items():
    df_clientes[col] = df_clientes[col].fillna(valor)

# Converter valor do contrato para float
df_clientes["VL_TOTAL_CONTRATO"] = (
    df_clientes["VL_TOTAL_CONTRATO"]
    .replace("[R$ ]", "", regex=True)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .round(2)
)

# Faixas de Valor do Contrato
bins = [0, 10000, 50000, 100000, 500000, 1000000, np.inf]
labels = ["ATE 10K", "10K–50K", "50K–100K", "100K–500K", "500K–1M", "ACIMA DE 1M"]
df_clientes["VL_TOTAL_CONTRATO_FAIXA"] = pd.cut(df_clientes["VL_TOTAL_CONTRATO"], bins=bins, labels=labels)

# Converter data de assinatura para datetime
df_clientes["DT_ASSINATURA_CONTRATO"] = pd.to_datetime(df_clientes["DT_ASSINATURA_CONTRATO"], errors="coerce")

# Ano e mês de assinatura
df_clientes["ANO_ASSINATURA"] = df_clientes["DT_ASSINATURA_CONTRATO"].dt.year.astype('int64')
df_clientes["MES_ASSINATURA"] = df_clientes["DT_ASSINATURA_CONTRATO"].dt.month.astype('int64')

# Idade do contrato (em meses)
df_clientes["IDADE_CONTRATO_MESES"] = df_clientes["DT_ASSINATURA_CONTRATO"].apply(idade_contrato_meses)

# Renomear colunas para padrão snake_case
df_clientes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "DS_PROD": "produto",
    "DS_LIN_REC": "linha_receita",
    "CIDADE": "cidade",
    "DS_CNAE": "cnae",
    "DS_SEGMENTO": "segmento",
    "DS_SUBSEGMENTO": "subsegmento",
    "FAT_FAIXA": "faixa_faturamento",
    "MARCA_TOTVS": "marca_totvs",
    "MODAL_COMERC": "modalidade_comercial",
    "PAIS": "pais",
    "PERIODICIDADE": "periodicidade",
    "SITUACAO_CONTRATO": "situacao_contrato",
    "UF": "uf",
    "VL_TOTAL_CONTRATO": "valor_total_contrato",
    "VL_TOTAL_CONTRATO_FAIXA": "faixa_valor_total_contrato",
    "DT_ASSINATURA_CONTRATO": "data_assinatura_contrato",
    "ANO_ASSINATURA": "ano_assinatura",
    "MES_ASSINATURA": "mes_assinatura",
    "IDADE_CONTRATO_MESES": "idade_contrato_meses"
}, inplace=True)

# Reorganizar colunas
ordem_colunas = [
    "cliente_id", "produto", "linha_receita", "cnae", "segmento", "subsegmento",
    "faixa_faturamento", "marca_totvs", "modalidade_comercial", "periodicidade",
    "situacao_contrato", "cidade", "uf", "pais",
    "valor_total_contrato", "faixa_valor_total_contrato",
    "data_assinatura_contrato", "ano_assinatura", "mes_assinatura", "idade_contrato_meses"
]
df_clientes = df_clientes[ordem_colunas]

# Exportação da base tratada
df_clientes.to_csv(f"{path_processed}/dados_clientes.csv", index=False)

#### **Pré-processamento: contratacoes_ultimos_12_meses.csv**

In [98]:
# Leitura da base de dados "contratacoes_ultimos_12_meses.csv"
contratacoes = pd.read_csv(f"{path_raw}/contratacoes_ultimos_12_meses.csv", sep=";")

# Cópia da base mãe
df_contratacoes = contratacoes.copy(deep=True)

# Conversão do valor para float
df_contratacoes["VLR_CONTRATACOES_12M"] = (
    df_contratacoes["VLR_CONTRATACOES_12M"]
    .replace("[R$ ]", "", regex=True)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .round(2)
)

# Renomear colunas para padrão snake_case
df_contratacoes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "QTD_CONTRATACOES_12M": "qtd_contratacoes_12m",
    "VLR_CONTRATACOES_12M": "vlr_contratacoes_12m"
}, inplace=True)

# Reorganizar colunas
df_contratacoes = df_contratacoes[["cliente_id", "qtd_contratacoes_12m", "vlr_contratacoes_12m"]]

# Exportar base tratada
df_contratacoes.to_csv(f"{path_processed}/contratacoes_ultimos_12_meses.csv", index=False)